In [59]:
import base64
import requests
import os
from PIL import Image
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()
api_key = os.getenv("STABILITY_API_KEY")
new_api_key = os.getenv("ROMAN_STABILITY_KEY")

In [70]:
text_prompts = ["One crisp morning, as the mist clung to the treetops like a veil of enchantment, Leo set out on a new adventure. Armed with nothing but his curiosity and a sense of wonder, he ventured deeper into the woods than ever before. As he wandered, he stumbled upon a forgotten path, overgrown with ivy and tangled roots. Intrigued, Leo followed it, his heart pounding with excitement. The path led him deeper into the heart of the forest, where the trees seemed to whisper secrets to one another, and the air hummed with magic.", 
"Soon, Leo came upon a clearing bathed in golden light. At its center stood an ancient oak tree, its branches reaching towards the sky like outstretched arms. Beneath the tree, nestled among its roots, was a small wooden chest, weathered with age. With trembling hands, Leo opened the chest, his eyes widening in astonishment at what lay inside. It was a treasure unlike any he had ever seen—glittering jewels, ancient artifacts, and a map that seemed to beckon him on a grand adventure.", 
"Finally, after a long and perilous journey, Leo reached the end of the map. Before him stood a shimmering lake, its waters sparkling like liquid silver in the sunlight. And there, on the shore, lay the greatest treasure of all—a sense of belonging, a deep connection to the forest, and the knowledge that adventure awaited just beyond the next horizon. As Leo stood on the edge of the lake, gazing out at the world that lay beyond the forest, he knew that no matter where his adventures took him, the forest would always be his home—a place of magic, mystery, and endless possibility."]

## V 2 beta version

### Text to image

In [46]:
def generate_image_from_text(text_prompts, output_path_images):
    
    url = f"https://api.stability.ai/v2beta/stable-image/generate/core"
    
    headers={
        "authorization": f"Bearer {api_key}",
        "accept": "image/*" #  "application/json"
    }
    
    body = {
        "prompt": text_prompts,
        "style_preset": "cinematic",
        "output_format": "png",
    }
    
    response = requests.post(url, headers=headers, files={"none": ''}, data=body,)

    #image_path = os.path.join(output_directory, f'{book_name}.png')
    
    file_name_path = f"{output_path_images}Trial_Image.png"

    if response.status_code == 200:
        with open(file_name_path, 'wb') as file:
            file.write(response.content)
    else:
        raise Exception(str(response.json()))
    
    return file_name_path

In [47]:
def resize_image(input_path, width = 768 , height = 768):

    # Load the image
    img = Image.open(input_path)

    # Resize the image
    img_resized = img.resize((width, height))

    # Save the resized image
    img_resized.save(input_path)


### Video Generation  

In [48]:
def get_generation_id(api_key, image_path, cfg_scale, motion_bucket_id):
    """
    cfg_scale [1, 10]: How strongly the video sticks to the original image.
    Use lower values to allow the model more freedom to make changes and higher values to correct motion distortions.

    motion_bucket_id [1, 255]: Lower values generally result in less motion in the output video, while higher values generally result in more motion.
    """

    url = f"https://api.stability.ai/v2beta/image-to-video"
    
    headers = {"authorization": f"Bearer {api_key}"}
    
    file = {"image": open(image_path, "rb")}
    
    data = {
        "seed": 0,
        "cfg_scale": cfg_scale,
        "motion_bucket_id": motion_bucket_id
    }
    
    response = requests.post(url, headers=headers, files=file, data=data,)
    
    #print("Generation ID:", response.json().get('id'))
    
    return response.json().get('id')

### Download Video

In [73]:
def download_generated_video(api_key, generation_id, output_path_video, retries=6, wait_time=10):
    
    url = f"https://api.stability.ai/v2beta/image-to-video/result/{generation_id}"
    
    headers = {
        'accept': "video/*",  # Use 'application/json' to receive base64 encoded JSON
        'authorization': f"Bearer {api_key}"
    }

    file_name_path = f"{output_path_video}Trial_Video.mp4"
    
    for attempt in range(retries):
        response = requests.request("GET", url, headers=headers) # , timeout=(5, 14)

        if response.status_code == 200:
            print("Generation complete!")
            with open(file_name_path, 'wb') as file:
                file.write(response.content)
            return  # Success, exit the loop

        elif response.status_code == 202:
            print(f"Generation in-progress, retrying attempt {attempt+1} of {retries} in {wait_time} seconds.")
            time.sleep(wait_time)
        else:
            raise Exception(str(response.json()))

    raise Exception("Failed to download video after all retries.")

### Final Function

In [68]:
def generate_and_download_video(api_key, text_prompts, book_name, cfg_scale, motion_bucket_id):

    """
    Generate image and animate it via Stability AI API. 
    Creates folder "book_name" with "book_name/Images" and "book_name/Videos".
    Stores created images and animation to corresponding folders.
    
    :param api_key: Stability AI API key
    
    :param text_prompts: Text to generate image from (e.g text_prompts = [ prompt_1, prompt_2, ...])
    
    :param book_name: Book name (e.g. book_name = "Harry")
    
    :param cfg_scale: [1, 10] How strongly the video sticks to the original image. 
    Use lower values to allow the model more freedom to make changes and higher values to correct motion distortions.
    
    :param motion_bucket_id: [1, 255] Lower values generally result in less motion in the output video, while higher values generally result in more motion. 
    """

    output_path_images = f"{book_name}/Images/"
    os.makedirs(output_path_images, exist_ok=True)

    output_path_video = f"{book_name}/Video/"
    os.makedirs(output_path_video, exist_ok=True)
    
    image_name = generate_image_from_text(text_prompts, output_path_images)
    
    resize_image(image_name, width = 768 , height = 768)
    
    video_id = get_generation_id(api_key, image_name, cfg_scale, motion_bucket_id)
    
    download_generated_video(api_key, video_id, output_path_video)

In [74]:
generate_and_download_video(api_key, text_prompts[2], "Final_test_Henry", cfg_scale = 1.8, motion_bucket_id = 127)

Generation in-progress, retrying attempt 1 of 6 in 10 seconds.
Generation in-progress, retrying attempt 2 of 6 in 10 seconds.
Generation in-progress, retrying attempt 3 of 6 in 10 seconds.
Generation in-progress, retrying attempt 4 of 6 in 10 seconds.
Generation complete!
